<a href="https://colab.research.google.com/github/mshinko/Research-project/blob/main/Independent_Study2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install datasets transformers rouge-score nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 23.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.9 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=dbb51aef2d00ba95e844305c02cd5aa1f697f478247314cf24cf2ce302a81cc1
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [21]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.1 MB/s eta 0:00:00


In [23]:
pip install accelerate -u


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


In [3]:
import transformers

print(transformers.__version__)

4.33.2


In [4]:
model_checkpoint = "t5-small"

In [5]:
pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.7 MB/s eta 0:00:00


In [6]:
pip install jsonlines

In [7]:
import jsonlines
import pandas as pd
from google.colab import files

# Use triple-quoted strings to preserve the formatting
original_text = """
Virtual Reality Sports Seating (VRSS) MOKRs
February 2022 - May 2022

Mission
VRSS aims to provide an immersive virtual reality experience to sports enthusiasts who would like to watch their favorite sports teams play. There would be no need for sports enthusiasts to travel to different arenas. In an ever-changing world where people are much more careful about contracting infectious diseases, this would provide fans with the reassurance that they could have an immersive experience from the comfort of their own homes - or wherever they would like to watch. VRSS will provide services for all types of sporting events from the high-school level, through college, to the professional level.

Objectives
During the next four months, VRSS will:
1. Investigate possible competitors - Understand what sports teams and streaming providers have already accomplished in the world of virtual reality and how VRSS can become a leader in the market.
2. Define intended customers - Identify the intended target market for this service. Create 2 customer profiles that represent the target market so that 10 adequate customer needs can be developed.
3. Identify a possible streaming method - Figure out the best way to record, stream, and allow an audience of 50+ people to watch a sports game using a few, budget-friendly video cameras.
4. Develop and launch a prototype website - Develop and update a prototype site based on customer needs that is easy to use and provides access to a VR video stream.
5. Apply for funding to expand - Acquire necessary resources to continue to grow. Explore which national markets will offer the best opportunity for expansion.

Page 1

Key Results

Investigate
possible
competitors

Generate a list of possible competitors to VRSS service.
Designate which competitors are offering, will-be offering, or do not currently offer VR streaming services.
Identify the successes and failures of the competitors that are offering VR streaming services.
Possibly contact/research companies to see what products are currently being used to support this feat.

Define intended
customers

Have associates interview with at least 5 friends and/or members of extended family to determine the likelihood of VR use based on age group.
Rank the receptiveness and likelihood of VR use on a 1-10 scale based on age range.
Determine a target market and develop three theoretical customer profiles that describe the intended customers.
Create a set of customer needs and target specifications based on the target market.

Identify a
possible
streaming
method

Visit websites such as GoPro to research the available VR-capable video cameras.
Research VR streaming capabilities to websites.
Study different types of arenas to determine the best possible locations to set up the video cameras.
Decide upon one video camera to move forward with for the company (or possibly more depending on each camera’s capabilities).
Decide upon different payment bundles for video streaming and viewing options.

Develop and
launch a
prototype
website

Research different free website options (i.e build from scratch using HTML or use a pre-built template).
Develop a website flow strategy based on prioritization and customer input.
Brainstorm multiple website design ideas to present to the class and receive constructive feedback.

Page 2

Decide upon one website design and attempt to stream a VR video to the website.

Apply for
funding
to expand

Create a preliminary business plan that outlines the tools needed to continue initial product development.
Attend at least one VT research event this semester and pitch the idea to more potential investors.
Have a plan to secure funding by the end of the semester.
"""

# Define the summary
summary = "Between February and May 2022, Virtual Reality Sports Seating (VRSS)..."

# Create a DataFrame
df = pd.DataFrame({'Original Text': [original_text], 'Summary': [summary]})

# Print the DataFrame
print(df)

                                       Original Text  \
0  \nVirtual Reality Sports Seating (VRSS) MOKRs\...   

                                             Summary  
0  Between February and May 2022, Virtual Reality...  


In [8]:
from datasets import load_dataset
import evaluate

metric = evaluate.load("rouge")

In [9]:
df['Original Text'][0]

'\nVirtual Reality Sports Seating (VRSS) MOKRs\nFebruary 2022 - May 2022\n\nMission\nVRSS aims to provide an immersive virtual reality experience to sports enthusiasts who would like to watch their favorite sports teams play. There would be no need for sports enthusiasts to travel to different arenas. In an ever-changing world where people are much more careful about contracting infectious diseases, this would provide fans with the reassurance that they could have an immersive experience from the comfort of their own homes - or wherever they would like to watch. VRSS will provide services for all types of sporting events from the high-school level, through college, to the professional level.\n\nObjectives\nDuring the next four months, VRSS will:\n1. Investigate possible competitors - Understand what sports teams and streaming providers have already accomplished in the world of virtual reality and how VRSS can become a leader in the market.\n2. Define intended customers - Identify the i

In [10]:
metric

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [12]:
if model_checkpoint in ["google/t5-small-lm-adapt", "t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
  prefix = "summarize: "
else:
  prefix =""

In [13]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(row):
    doc = row["Original Text"]
    summary = row["Summary"]

    inputs = [prefix + doc]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(summary, max_length=max_target_length, truncation=True)

    # Return a dictionary containing the model inputs and labels
    return {
        "input_ids": model_inputs["input_ids"],
        "attention_mask": model_inputs["attention_mask"],
        "labels": labels["input_ids"],
    }


In [14]:
# Apply the preprocess_function to your DataFrame
preprocessed_data = df.apply(preprocess_function, axis=1)
# Print the preprocessed data
print(preprocessed_data)

0    {'input_ids': [[21603, 10, 11143, 23963, 5716,...
dtype: object


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [16]:
tokenized_datasets = df.apply(preprocess_function, axis=1)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [17]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [24]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16 = True,
    push_to_hub=False,
)

ImportError: ignored